### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program
>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

from datetime import datetime, time  # Import time-related modules
from enum import Enum  # Import Enum for availability status
from typing import Dict, List  # Import type hints

class Availability(Enum):
    """Enum for caregiver availability status"""
    PREFERRED = "preferred"     # Caregiver prefers this shift
    AVAILABLE = "available"     # Caregiver can work this shift
    UNAVAILABLE = "unavailable" # Caregiver cannot work this shift

class Shift:
    """Class representing a work shift"""
    AM_START = time(7, 0)   # Morning shift start time: 7:00 AM
    AM_END = time(13, 0)    # Morning shift end time: 1:00 PM
    PM_START = time(13, 0)  # Afternoon shift start time: 1:00 PM
    PM_END = time(19, 0)    # Afternoon shift end time: 7:00 PM

    def __init__(self, is_am: bool):
        self.is_am = is_am  # Boolean flag for AM/PM shift
        self.start_time = self.AM_START if is_am else self.PM_START  # Set start time based on shift
        self.end_time = self.AM_END if is_am else self.PM_END       # Set end time based on shift

    def __str__(self):
        shift_name = "AM" if self.is_am else "PM"  # Get shift name (AM/PM)
        return f"{shift_name} Shift ({self.start_time.strftime('%I:%M %p')} - {self.end_time.strftime('%I:%M %p')})"  # Format shift times

class Caregiver:
    """Class representing a caregiver"""
    def __init__(self, name: str, phone: str, email: str, pay_rate: float):
        self.name = name           # Caregiver's name
        self.phone = phone         # Caregiver's phone number
        self.email = email         # Caregiver's email
        self.pay_rate = pay_rate   # Hourly pay rate
        self.hours = 0             # Initialize worked hours to 0
        # Create default availability schedule - all shifts set to available
        self.availability = {
            'monday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE},
            'tuesday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE},
            'wednesday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE},
            'thursday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE},
            'friday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE},
            'saturday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE},
            'sunday': {'am': Availability.AVAILABLE, 'pm': Availability.AVAILABLE}
        }

    def set_availability(self, day: str, shift: str, status: Availability):
        """Set availability for a specific day and shift"""
        day = day.lower()     # Convert day to lowercase
        shift = shift.lower() # Convert shift to lowercase
        if day not in self.availability:  # Validate day input
            raise ValueError(f"Invalid day: {day}")
        if shift not in ['am', 'pm']:     # Validate shift input
            raise ValueError(f"Invalid shift: {shift}")
        if not isinstance(status, Availability):  # Validate status input
            raise ValueError(f"Invalid availability status: {status}")
        
        self.availability[day][shift] = status  # Set the availability status

    def get_availability(self, day: str, shift: str) -> Availability:
        """Get availability for a specific day and shift"""
        day = day.lower()     # Convert day to lowercase
        shift = shift.lower() # Convert shift to lowercase
        return self.availability[day][shift]  # Return availability status

    def set_weekly_availability(self, day: str, status: Availability):
        """Set availability for both shifts on a specific day"""
        self.set_availability(day, 'am', status)  # Set morning availability
        self.set_availability(day, 'pm', status)  # Set afternoon availability

    def add_hours(self, hours: float):
        """Add worked hours to caregiver's total"""
        self.hours += hours  # Add hours to total

    def __str__(self):
        return f"Caregiver: {self.name}\nContact: {self.phone}, {self.email}\nPay Rate: ${self.pay_rate}/hr"  # Format caregiver info

class CaregiverManager:
    """Class to manage multiple caregivers"""
    def __init__(self):
        self.caregivers: List[Caregiver] = []  # Initialize empty list of caregivers

    def add_caregiver(self, caregiver: Caregiver):
        """Add a new caregiver to the system"""
        self.caregivers.append(caregiver)  # Add caregiver to list

    def remove_caregiver(self, caregiver_name: str):
        """Remove a caregiver from the system by name"""
        self.caregivers = [cg for cg in self.caregivers if cg.name != caregiver_name]  # Filter out caregiver by name

    def get_caregiver(self, name: str) -> Caregiver:
        """Get a caregiver by name"""
        for caregiver in self.caregivers:  # Search through caregivers
            if caregiver.name.lower() == name.lower():  # Case-insensitive name comparison
                return caregiver
        raise ValueError(f"Caregiver not found: {name}")  # Raise error if not found

    def get_available_caregivers(self, day: str, shift: str) -> List[Caregiver]:
        """Get list of caregivers available for a specific day and shift"""
        return [cg for cg in self.caregivers  # Return list of caregivers
                if cg.get_availability(day, shift) in 
                [Availability.AVAILABLE, Availability.PREFERRED]]  # Check if available or preferred

    def get_preferred_caregivers(self, day: str, shift: str) -> List[Caregiver]:
        """Get list of caregivers who prefer a specific day and shift"""
        return [cg for cg in self.caregivers  # Return list of caregivers
                if cg.get_availability(day, shift) == Availability.PREFERRED]  # Check if preferred

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> Example: [INST326_Fall2024/Projects/Project03](https://github.com/sdempwolf/INST326_Fall_2024/tree/main/Projects/Project03)
>
> Edit the link code below with your information, then run this cell. Test the link! It should take you to your GitHub project repository.
> [external link text](http://url_here)

In [6]:
# Solution - enter your code solution below


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS